In [1]:
""" Imports """
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks
import sys

# Manual Work - no automation

In [2]:
""" Pull Data Manually """
ch1data = pd.read_csv("LtoutputFiles/output5ch1_05_03_2023.txt", sep="\t")
ch4data = pd.read_csv("LtoutputFiles/output5ch4_05_03_2023.txt", sep="\t")

In [7]:
"""
Formatting 
- Name Columns 
- Create Data frame
    - Quick plot: <dataframe>.plot()
- Truncate: not needed (?)
- Array assignment 
"""

# Name columns 
ch1data.columns =["Time", "V(compout)-CH1"]
ch4data.columns =["Time", "V(compout)-CH4"]

# Create data frame 
ch1 = pd.DataFrame(ch1data)
ch4 = pd.DataFrame(ch4data)

# Check for shorter file and truncate 
#   - LTspice always outputs diferent length files 

# # print(f"len ch1: {len(ch1)}\nlen ch4: {len(ch4)}")
# if len(ch1) < len(ch4): 
#     ch1 = ch1.truncate(after=len(ch1)-1) # smallest less 1 bc `after` adds a value
#     ch4 = ch4.truncate(after=len(ch1)-1)
# else: 
#     ch1 = ch1.truncate(after=len(ch4)-1)
#     ch4 = ch4.truncate(after=len(ch4)-1)
# # print(f"\nTruncation:\nlen ch1: {len(ch1)}\nlen ch4: {len(ch4)}")

# Naming and floats
ch1t = ch1["Time"].to_numpy()
ch4t = ch4["Time"].to_numpy()
ch1V = ch1["V(compout)-CH1"].to_numpy()
ch4V = ch4["V(compout)-CH4"].to_numpy()